In [1]:
%load_ext sparksql_magic
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col

spark = SparkSession.builder.appName("CreateTable").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/11 16:47:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.json("/data/*.json.gz")
# Filter for IssuesEvent
df = df.filter(df["type"] == "IssuesEvent")
df = df.select(
    col("actor"),
    col("created_at"),
    col("id").alias("event_id"),
    col("org"),
    col("payload.action"),
    col("payload.issue")
)

25/03/11 16:47:59 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


## 🌊🐳🧊 Example of creating an Iceberg Table

In [7]:
%%sql
CREATE TABLE IF NOT EXISTS iceberg.github_issues (
    actor STRING,
    created_at TIMESTAMP,
    event_id STRING,
    org STRING,
    action STRING,
    issue STRING
) USING ICEBERG
PARTITIONED BY (days(created_at));

++
||
++
++

## To insert data:
this will fail because the data types are wrong

In [18]:
# df.write.format("iceberg") \
#     .mode("overwrite") \
#     .saveAsTable("iceberg.github_issues")

In [10]:
%%sql
DESCRIBE TABLE iceberg.github_issues;

col_name,data_type,comment
actor,string,None
created_at,timestamp,None
event_id,string,None
org,string,None
action,string,None
issue,string,None
,,
# Partitioning,,
Part 0,days(created_at),


In [15]:
%%sql
DESCRIBE FORMATTED iceberg.github_issues;

col_name,data_type,comment
actor,string,None
created_at,timestamp,None
event_id,string,None
org,string,None
action,string,None
issue,string,None
,,
# Partitioning,,
Part 0,days(created_at),
,,


In [16]:
%%sql
INSERT INTO iceberg.github_issues 
VALUES ('test_actor', TIMESTAMP '2024-03-11 12:00:00', '12345', 'test_org', 'opened', 'test_issue');


++
||
++
++

In [17]:
%%sql 
select * from iceberg.github_issues 

actor,created_at,event_id,org,action,issue
test_actor,2024-03-11 12:00:00,12345,test_org,opened,test_issue
